In [1]:
import nltk
import pandas as pd
nltk.download("stopwords")
nltk.download('punkt')
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfTransformer


import string
import re
from google.colab import drive
drive.mount('/content/drive/')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def clean(arg):
    arg=arg.lower()
    arg=arg.replace("_", " ")
    arg=re.sub("","",arg)
    # no sonderzeichen 
    arg=re.sub("(\\d|\\W)+"," ",arg)
    return arg

# order by 
def sort_tfidf(mat):
    tuples = zip(mat.col, mat.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def topn(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/cloudcomputing/cleaned_data_with_keywords.csv", sep="\t").drop(columns=["keywords"])
df["arg"] = df["premises"] + " " + df["conclusion"]

In [ ]:
%%time
arg = df["arg"].apply(lambda arg: clean(str(arg)))

In [ ]:
%%time
cv = CountVectorizer(strip_accents = 'unicode', # works 
                                stop_words = stwords, # works
                                lowercase = True, # works
                                max_df = 0.85) # works
word_count_vector=cv.fit_transform(arg)

In [ ]:
%%time
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
%%time
feature_names=cv.get_feature_names()
kw = []
for doc in args:
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
    sorted_items=sort_tfidf(tf_idf_vector.tocoo())
    keywords=topn(feature_names,sorted_items,10)
    kw.append(list(keywords.keys()))



In [ ]:
count = [len(k) for k in kw]
df["keywords"] = kw
df["key_count"] = count

In [ ]:
%%time
synonyms = []
for keywords in kw:   
  synonyms.append([list(set([i.lemmas()[0].name() for i in wordnet.synsets(word)])) for word in keywords])

syn = []
for s in synonyms:
  syn.append([item for sublist in s for item in sublist])

In [ ]:
count = [len(set(s)) for s in syn]
syn = [" ".join(set(s)) for s in syn]
df["synonyms"] = syn
df["syn_count"] = count
df.to_csv("/content/drive/MyDrive/cloudcomputing/cleaned_data_with_key_syn.csv", "\t", index=False)


In [ ]:
df["premises"] = df["premises"] + " " + df["synonyms"]
df = df[["premises", "id", "conclusion", "context.sourceUrl", "stance"]]
df.to_csv("/content/drive/MyDrive/cloudcomputing/data_naiveexpansion.csv", "\t", index=False)